In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
import json
from pprint import pprint
from datetime import datetime
from pathlib import Path

In [2]:
# #csv sports_io data pull if the api is not utilized

# data_dict = {}
# csv_path = f'../data/sports_io/'

# for i in os.listdir(csv_path):
#     data_df = pd.read_csv(csv_path + i)
    
#     data_dict[i.split('.')[0]] = data_df

# print(data_dict.keys())

In [5]:
#sports_io nba documentation link:  https://fantasydata.com/api/api-documentation/nba#
#data_set_list = ['GamesByDate', 'GameOddsByDate', 'GameOddsLineMovement', 'Schedules', 'Stadiums', 'TeamGameStatsByDate', 'TeamSeasonStats']
#odds_data_set_dict = {'GamesByDate' : 'YYYY-MM-DD', 'GameOddsByDate' : 'YYYY-MM-DD', 'GameOddsLineMovement' : 'GameID', 'Schedules' : 'Season', 'Stadiums' : NaN, 'TeamGameStatsByDate' : 'YYYY-MM-DD', 'TeamSeasonStats' : 'Season'}

#assign sports_io api key
load_dotenv()
sports_io_api_key = os.getenv('sports_io_api_key')


#assign sports_io url to variable that can be pulled with different parameters
def sports_io_api_pull(data_set, data_set_parameter, sports_io_api_key):
    sports_io_odds_url = f"https://api.sportsdata.io/api/nba/odds/json/{data_set}/{data_set_parameter}?key={sports_io_api_key}"

    response_data = pd.read_json(sports_io_odds_url)

    # response_df = pd.DataFrame(response_data)

    return response_data

In [6]:
data_set = 'GamesByDate'
date_list = pd.date_range(start = '1/2/2020', periods = 5).strftime(('%Y-%m-%d'))

nba_odds_list = []

for i in date_list:

    nba_odds_data = sports_io_api_pull(data_set, i, sports_io_api_key)
    nba_odds_list.append(nba_odds_data)

nba_odds_df = pd.concat(nba_odds_list, axis = 0).drop(columns = 'Quarters')

In [7]:
nba_odds_df.head()

,GameID,Season,SeasonType,Status,Day,DateTime,AwayTeam,HomeTeam,AwayTeamID,HomeTeamID,StadiumID,AwayTeamScore,HomeTeamScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,HomeRotationNumber,AwayRotationNumber,NeutralVenue
0,14411,2020,1,Final,2020-01-02T00:00:00,2020-01-02 19:00:00,CHA,CLE,2,12,12,109,106,-1.5,212.5,105,-125,546.0,545.0,False
1,14412,2020,1,Final,2020-01-02T00:00:00,2020-01-02 19:00:00,DEN,IND,20,13,13,124,116,1.0,208.5,-113,-107,548.0,547.0,False
2,14413,2020,1,Final,2020-01-02T00:00:00,2020-01-02 19:30:00,TOR,MIA,10,4,4,76,84,-5.5,214.5,177,-217,550.0,549.0,False
3,14414,2020,1,Final,2020-01-02T00:00:00,2020-01-02 20:00:00,UTA,CHI,17,11,11,102,98,3.0,209.0,-155,130,NaN,NaN,False
4,14415,2020,1,Final,2020-01-02T00:00:00,2020-01-02 20:00:00,GS,MIN,26,16,16,84,99,-3.0,212.0,126,-150,552.0,551.0,False


In [6]:
data_set_ii = 'GameOddsLineMovement'
game_list = list(nba_odds_df['GameID'])

opening_lines_list = []

for i in game_list:

    line_movement_data = sports_io_api_pull(data_set_ii, i, sports_io_api_key)

    line_movement_data['PregameOdds'] = line_movement_data['PregameOdds'].apply(pd.DataFrame)
    line_movement_data['opening_lines_home'] = line_movement_data['PregameOdds'].apply(lambda x: x.loc[x['Created'] == x['Created'].min(), 'HomePointSpread'])
    line_movement_data = line_movement_data[['GameId', 'opening_lines_home']]

    opening_lines_list.append(line_movement_data)

opening_lines_df = pd.concat(opening_lines_list).reset_index(drop = True)

In [8]:
nba_odds_df.head()

,GameID,Season,SeasonType,Status,Day,DateTime,AwayTeam,HomeTeam,AwayTeamID,HomeTeamID,StadiumID,AwayTeamScore,HomeTeamScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,HomeRotationNumber,AwayRotationNumber,NeutralVenue
0,14411,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:00:00,CHA,CLE,2,12,12,109,106,-1.5,212.5,105,-125,546.0,545.0,False
1,14412,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:00:00,DEN,IND,20,13,13,124,116,1.0,208.5,-113,-107,548.0,547.0,False
2,14413,2020,1,Final,2020-01-02T00:00:00,2020-01-02T19:30:00,TOR,MIA,10,4,4,76,84,-5.5,214.5,177,-217,550.0,549.0,False
3,14414,2020,1,Final,2020-01-02T00:00:00,2020-01-02T20:00:00,UTA,CHI,17,11,11,102,98,3.0,209.0,-155,130,NaN,NaN,False
4,14415,2020,1,Final,2020-01-02T00:00:00,2020-01-02T20:00:00,GS,MIN,26,16,16,84,99,-3.0,212.0,126,-150,552.0,551.0,False
